In [1]:
spark

Intitializing Scala interpreter ...

Spark Web UI available at http://172.30.53.50:4040
SparkContext available as 'sc' (version = 3.3.2, master = local[*], app id = local-1679032734263)
SparkSession available as 'spark'


res0: org.apache.spark.sql.SparkSession = org.apache.spark.sql.SparkSession@1a7ec20f


# 8.1 조인 표현식
- 스파크는 왼쪽과 오른쪽 데이터셋에 있는 하나 이상의 키값을 비교하고 왼쪽 데이터셋과 오른쪽 데이터셋의 결합 여부를 결정하는 조인 표현식의 평가 결과에 따라 두 개의 데이터셋을 조인
- 가장 많이 사용하는 조인 표현식 : 왼쪽과 오른쪽 데이터셋에 지정된 키가 동일한지 비교하는 동등 조인(equi-join)
    - 키가 일치하면 스파크는 왼쪽과 오른쪽 데이터셋을 결합
    - 일치하지 않으면 데이터셋을 결합하지 않음
- 스파크는 일치하는 키가 없는 로우는 조인에 포함시키지 않음
- 동등 조인 뿐만 아니라 더 복잡한 조인 정책도 지원, 복합 데이터 타입을 조인에 사용할 수도 있음
    - e.g. 배열 타입의 키에 조인할 키가 존재하는지 확인해 조인을 수행

# 8.2 조인 타입
- 조인 표현식은 두 로우의 조인 여부를 결정
- 조인 타입은 결과 데이터셋에 어떤 데이터가 있어야 하는지 결정
    - 내부 조인(inner join) : 왼쪽고 오른쪽 데이터셋에 키가 있는 로우를 유지
    - 외부 조인(outer join) : 왼쪽이나 오른쪽 데이터셋에 키가 있는 로우를 유지
    - 왼쪽 외부 조인(left outer join) : 왼쪽 데이터셋에 키가 있는 로우를 유지
    - 오른쪽 외부 조인(right outer join) : 오른쪽 데이터셋에 키가 있는 로우를 유지
    - 왼쪽 세미 조인(left semi join) : 왼쪽 데이터셋의 키가 오른쪽 데이터셋에 있는 경우에는 키가 일치하는 왼쪽 데이터셋만 유지
    - 왼쪽 안티 조인(left anti join) : 왼쪽 데이터셋의 키가 오른쪽 데이터셋에 없는 경우에는 키가 일치하지 않는 왼쪽 데이터셋만 유지
    - 자연 조인(natural join) : 두 데이터셋에서 동일한 이름을 가진 커럼을 암시적(implicit)으로 결합하는 조인
    - 교차 조인(cross join) 또는 카테시안 조인(Cartesion join) : 왼쪽 데이터셋의 모든 로우와 오른쪽 데이터셋의 모든 로우를 조합

In [4]:
val person = Seq(
    (0, "Bill Chambers", 0, Seq(100)),
    (1, "Matei Zaharia", 1, Seq(500, 250, 100)),
    (2, "Michael Armbrust", 1, Seq(250, 100)))
.toDF("id", "name", "graduate_program", "spark_status")

person: org.apache.spark.sql.DataFrame = [id: int, name: string ... 2 more fields]


In [5]:
val graduateProgram = Seq(
    (0, "Masters", "School of Information", "UC Berkelye"),
    (2, "Masters", "EECS", "UC Berkely"),
    (1, "Ph.D", "EECS", "UC Berkely"))
.toDF("id", "degree", "department", "school")

graduateProgram: org.apache.spark.sql.DataFrame = [id: int, degree: string ... 2 more fields]


In [6]:
val sparkStatus = Seq(
    (500, "Vice President"),
    (250, "PMC Member"),
    (100, "Contributor"))
.toDF("id", "status")

sparkStatus: org.apache.spark.sql.DataFrame = [id: int, status: string]


In [7]:
person.createOrReplaceTempView("person")
graduateProgram.createOrReplaceTempView("graduateProgram")
sparkStatus.createOrReplaceTempView("sparkStatus")

# 8.3 내부 조인
- 내부 조인은 DataFrame이나 테이블에 존재하는 키를 평가하고 참(true)으로 평가되는 로우만 결합

In [9]:
val joinExpression = person.col("graduate_program") === graduateProgram.col("id")

joinExpression: org.apache.spark.sql.Column = (graduate_program = id)


- 두 DataFrame 모두에 키가 존재하지 않으면 결과 DataFrame에서 볼 수 없음

In [10]:
val wrongJoinExpression = person.col("name") === graduateProgram.col("school")

wrongJoinExpression: org.apache.spark.sql.Column = (name = school)


In [11]:
person.join(graduateProgram, joinExpression).show()

+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkelye|
|  2|Michael Armbrust|               1|     [250, 100]|  1|   Ph.D|                EECS| UC Berkely|
|  1|   Matei Zaharia|               1|[500, 250, 100]|  1|   Ph.D|                EECS| UC Berkely|
+---+----------------+----------------+---------------+---+-------+--------------------+-----------+



# 8.4 외부 조인
- 외부 조인은 DataFrame이나 테이블에 존재하는 키를 평가하여 참이나 거짓으로 평가한 로우를 포함(그리고 조인)함
- 왼쪽이나 오른쪽 DataFrame에 일치하는 로우가 없다면 스파크는 해당 위치에 null을 삽입

In [12]:
var joinType = "outer"

joinType: String = outer


In [13]:
person.join(graduateProgram, joinExpression, joinType).show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkelye|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|   Ph.D|                EECS| UC Berkely|
|   2|Michael Armbrust|               1|     [250, 100]|  1|   Ph.D|                EECS| UC Berkely|
|null|            null|            null|           null|  2|Masters|                EECS| UC Berkely|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+



# 8.5 왼쪽 외부 조인
- 왼쪽 외부 조인은 DataFrame이나 테이블에 존재하는 키를 평가
- 왼쪽 DataFrame의 모든 로우와 왼쪽 DataFrame과 일치하는 오른쪽 DataFrame의 로우를 함께 포함
- 오른쪽 DataFrame에 일치하는 로우가 없다면 스파크는 해당 위치에 null을 삽입

In [14]:
joinType = "left_outer"
graduateProgram.join(person,joinExpression,joinType).show()

+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
| id| degree|          department|     school|  id|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkelye|   0|   Bill Chambers|               0|          [100]|
|  2|Masters|                EECS| UC Berkely|null|            null|            null|           null|
|  1|   Ph.D|                EECS| UC Berkely|   2|Michael Armbrust|               1|     [250, 100]|
|  1|   Ph.D|                EECS| UC Berkely|   1|   Matei Zaharia|               1|[500, 250, 100]|
+---+-------+--------------------+-----------+----+----------------+----------------+---------------+



joinType: String = left_outer


# 8.6 오른쪽 외부 조인
- 오른쪽 외부 조인은 DataFrame이나 테이블에 존재하는 키를 평가
- 오른쪽 DataFrame의 모든 로우와 오른쪽 DataFrame과 일치하는 왼쪽 DataFrame의 로우를 함께 포함
- 왼쪽 DataFrame에 일치하는 로우가 없다면 스파크는 해당 위치에 null을 삽입

In [15]:
joinType = "right_outer"
person.join(graduateProgram, joinExpression, joinType).show()

+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|  id|            name|graduate_program|   spark_status| id| degree|          department|     school|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+
|   0|   Bill Chambers|               0|          [100]|  0|Masters|School of Informa...|UC Berkelye|
|null|            null|            null|           null|  2|Masters|                EECS| UC Berkely|
|   2|Michael Armbrust|               1|     [250, 100]|  1|   Ph.D|                EECS| UC Berkely|
|   1|   Matei Zaharia|               1|[500, 250, 100]|  1|   Ph.D|                EECS| UC Berkely|
+----+----------------+----------------+---------------+---+-------+--------------------+-----------+



joinType: String = right_outer


# 8.7 왼쪽 세미 조인
- 세미 조인은 오른쪽 DataFrame의 어떤 값도 포함하지 않기 때문에 다른 조인 타입과는 약간 다름
- 단지 두 번재 DataFrame은 값이 존재하는지 확인하기 위해 값만 비교하는 용도로 사용
- 만약 값이 존재한다면 왼쪽 DataFrame에 중복 키가 존재하더라도 해당 로우는 결과에 포함됨
- 왼쪽 세미 조인은 기존 조인 기능과는 달리 DataFrame의 필터 정도로 볼 수 있음

In [16]:
joinType = "left_semi"
graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkelye|
|  1|   Ph.D|                EECS| UC Berkely|
+---+-------+--------------------+-----------+



joinType: String = left_semi


In [20]:
val gradProgram2 = graduateProgram.union(Seq(
    (0, "Masters", "Duplicated row", "Duplicated School")).toDF())
gradProgram2.createOrReplaceTempView("gradProgram2")

gradProgram2: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [id: int, degree: string ... 2 more fields]


# 8.8 왼쪽 안티 조인
- 왼쪽 세미 조인의 반대 개념
- 왼쪽 세미 조인처럼 오른쪽 DataFrame의 어떤 값도 포함하지 않음
- 단지 두 번째 DataFrame은 값이 존재하는지 확인하기 위해 값만 비교하는 용도로 사용
- 하지만 두 번째 DataFrame에 존재하는 값을 유지하는 대신 두 번재 DataFrame에서 관련된 키를 찾을 수 없는 로우만 결과에 포함
- 안티 조인은 SQL의 NOT IN과 같은 스타일의 필터로 볼 수 있음

In [21]:
joinType = "left_anti"
graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+----------+----------+
| id| degree|department|    school|
+---+-------+----------+----------+
|  2|Masters|      EECS|UC Berkely|
+---+-------+----------+----------+



joinType: String = left_anti


# 8.9 자연 조인
- 조인하려는 컬럼을 암시적으로 추정
    - 즉, 일치하는 컬럼을 찾고 그 결과를 반환
- 왼쪽과 오른쪽 그리고 외부 자연 조인을 사용할 수 있음

# 8.10 교차 조인(카테시안 조인)
- 조건절을 기술하지 않은 내부 조인
- 왼쪽 DataFrame의 모든 로우를 오른쪽 DataFrame의 모든 로우와 결합
    - 엄청난 수를 가진 DataFrame이 생성될 수 있음
    - 따라서 반드시 키워드를 이용해 교차 조인을 수행한다는 것을 명시적으로 선언해야 함


In [22]:
joinType = "cross"
graduateProgram.join(person, joinExpression, joinType).show()

+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
| id| degree|          department|     school| id|            name|graduate_program|   spark_status|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+
|  0|Masters|School of Informa...|UC Berkelye|  0|   Bill Chambers|               0|          [100]|
|  1|   Ph.D|                EECS| UC Berkely|  2|Michael Armbrust|               1|     [250, 100]|
|  1|   Ph.D|                EECS| UC Berkely|  1|   Matei Zaharia|               1|[500, 250, 100]|
+---+-------+--------------------+-----------+---+----------------+----------------+---------------+



joinType: String = cross


# 8.11 조인 사용 시 문제점
## 8.11.1 복합 데이터 타입의 조인
- 불리언을 반환하는 모든 표현식은 조인 표현식으로 간주할 수 있음

In [23]:
import org.apache.spark.sql.functions.expr

import org.apache.spark.sql.functions.expr


In [24]:
person.withColumnRenamed("id", "personId").join(sparkStatus, expr("array_contains(spark_status, id)")).show()

+--------+----------------+----------------+---------------+---+--------------+
|personId|            name|graduate_program|   spark_status| id|        status|
+--------+----------------+----------------+---------------+---+--------------+
|       0|   Bill Chambers|               0|          [100]|100|   Contributor|
|       1|   Matei Zaharia|               1|[500, 250, 100]|500|Vice President|
|       1|   Matei Zaharia|               1|[500, 250, 100]|250|    PMC Member|
|       1|   Matei Zaharia|               1|[500, 250, 100]|100|   Contributor|
|       2|Michael Armbrust|               1|     [250, 100]|250|    PMC Member|
|       2|Michael Armbrust|               1|     [250, 100]|100|   Contributor|
+--------+----------------+----------------+---------------+---+--------------+



## 8.11.2 중복 컬럼명 처리
- 조인을 수행할 때 가장 까다로운 것 중 하나는 결과 DataFrame에서 중복된 컬럼명을 다루는 것
- DataFrame의 각 컬럼은 스파크 SQL 엔진인 카탈리스트 내에 고유 ID가 있음
- 고유 ID는 카탈리스트 내부에서만 사용할 수 있으며 직접 참조할 수 있는 값이 아님
- 그러므로 중복된 컬럼명이 존재하는 DataFrame을 사용할 때는 특정 컬럼을 참조하기 매우 어려움
- 이런 문제를 일으키는 두 가지 상황
    - 조인에 사용할 DataFrame의 특정 키가 동일한 이름을 가지며, 키가 제거되지 않도록 조인 표현식에 명시하는 경우
    - 조인 대상이 아닌 두 개의 컬럼이 동일한 이름을 가진 경우

In [25]:
val gradProgramDupe = graduateProgram.withColumnRenamed("id", "graduate_program")

val joinExpr = gradProgramDupe.col("graduate_program") === person.col("graduate_program")

gradProgramDupe: org.apache.spark.sql.DataFrame = [graduate_program: int, degree: string ... 2 more fields]
joinExpr: org.apache.spark.sql.Column = (graduate_program = graduate_program)


In [26]:
person.join(gradProgramDupe, joinExpr).show()

+---+----------------+----------------+---------------+----------------+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status|graduate_program| degree|          department|     school|
+---+----------------+----------------+---------------+----------------+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|               0|Masters|School of Informa...|UC Berkelye|
|  2|Michael Armbrust|               1|     [250, 100]|               1|   Ph.D|                EECS| UC Berkely|
|  1|   Matei Zaharia|               1|[500, 250, 100]|               1|   Ph.D|                EECS| UC Berkely|
+---+----------------+----------------+---------------+----------------+-------+--------------------+-----------+



In [27]:
person.join(gradProgramDupe, joinExpr).select("graduate_program").show()

org.apache.spark.sql.AnalysisException:  Reference 'graduate_program' is ambiguous, could be: graduate_program, graduate_program.

### 해결 방법 1: 다른 조인 표현식 사용
- 불리언 형태의 조인 표현식을 문자열이나 시퀀스 형태로 바꾸는 것
    - 이렇게 하면 조인을 할 때두 컬럼 중 하나가 자동으로 제거됨

In [28]:
person.join(gradProgramDupe, "graduate_program").select("graduate_program").show()

+----------------+
|graduate_program|
+----------------+
|               0|
|               1|
|               1|
+----------------+



### 해결 방법 2 : 조인 후 컬럼 제거
- 원본 DataFrame을 사용해 컬럼을 참조해야 함
- 조인 시 동일한 키 이름을 사용하거나 원본 DataFrame에 동일한 컬럼명이 존재하는 경우에 사용할 수 있음

In [30]:
person.join(gradProgramDupe, joinExpr).drop(person.col("graduate_program")).select("graduate_program").show()

+----------------+
|graduate_program|
+----------------+
|               0|
|               1|
|               1|
+----------------+



In [31]:
val joinExpr = person.col("graduate_program") === graduateProgram.col("id")
person.join(graduateProgram, joinExpr).drop(graduateProgram.col("id")).show()

+---+----------------+----------------+---------------+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status| degree|          department|     school|
+---+----------------+----------------+---------------+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|Masters|School of Informa...|UC Berkelye|
|  2|Michael Armbrust|               1|     [250, 100]|   Ph.D|                EECS| UC Berkely|
|  1|   Matei Zaharia|               1|[500, 250, 100]|   Ph.D|                EECS| UC Berkely|
+---+----------------+----------------+---------------+-------+--------------------+-----------+



joinExpr: org.apache.spark.sql.Column = (graduate_program = id)


### 해결 방법 3 : 조인 전 컬럼명 변경

In [32]:
val gradProgram3 = graduateProgram.withColumnRenamed("id", "grad_id")
val joinExpr = person.col("graduate_program") === gradProgram3.col("grad_id")
person.join(gradProgram3, joinExpr).show()

+---+----------------+----------------+---------------+-------+-------+--------------------+-----------+
| id|            name|graduate_program|   spark_status|grad_id| degree|          department|     school|
+---+----------------+----------------+---------------+-------+-------+--------------------+-----------+
|  0|   Bill Chambers|               0|          [100]|      0|Masters|School of Informa...|UC Berkelye|
|  2|Michael Armbrust|               1|     [250, 100]|      1|   Ph.D|                EECS| UC Berkely|
|  1|   Matei Zaharia|               1|[500, 250, 100]|      1|   Ph.D|                EECS| UC Berkely|
+---+----------------+----------------+---------------+-------+-------+--------------------+-----------+



gradProgram3: org.apache.spark.sql.DataFrame = [grad_id: int, degree: string ... 2 more fields]
joinExpr: org.apache.spark.sql.Column = (graduate_program = grad_id)


# 8.12 스파크의 조인 수행 방식
- 실행에 필요한 두 가지 핵심 전략
    1. 노드간 네트워크 통신 전략
    2. 노드별 연산 전략

## 8.12.1 네트워크 통신 전략
- 스파크는 조인 시 두 가지 클러스터 통신 방식을 활용
    - 셔플 조인 : 전체 노드 간 통신을 유발
    - 브로드캐스트 조인
- 이제부터는 사용자가 스파크에서 사용하는 테이블의 크기가 아주 크거나 아주 작다고 가정

### 큰 테이블과 큰 테이블 조인
- 하나의 큰 테이블을 다른 큰 테이블과 조인하면 셔플 조인이 발생
- 셔플 조인은 전체 노드 간 통신 발생. 그리고 조인에 사용한 특정 키나 키 집합을 어떤 노드가 가졌는지에 따라 해당 노드와 데이터를 공유
- 이런 통신 방식 때문에 네트워크는 복잡해지고 많은 자원을 사용 (특히 데이터가 잘 나뉘어 있지 않다면 더 심해짐)

### 큰 테이블과 작은 테이블 조인
- 테이블이 단일 워커 노드의 메모리 크기에 적합할 정도로 충분히 작은 경우 조인 연산을 최적화할 수 있음
- 브로드캐스트 조인이 훨씬 효율적
    - 작은 DataFrame을 클러스터의 전체 워커 노드에 복제(조인 프로세스 내내 전체 노드가 통신하는 현상을 방지)

### 아주 작은 테이블 사이의 조인
- 스파크가 결정하도록 내버려 두는 것이 좋음